This notebook is based on the article "**Operad Structure of Poset Matrices**"
Available in https://arxiv.org/abs/2401.06814 and the implementation of the algorithms in **Computational and Combinatorial Enumeration of Poset Matrices**(In Press).

In [ ]:
# import  packages
import numpy as np
import itertools
import more_itertools
import math

**isBinary** function determines if a matriz is binary (0,1).

In [ ]:
def isBinary(inputarray):
  return ((inputarray==0) | (inputarray==1)).all()

**isPosetMatrix** function determines if a binary triangular matrix is a poset.

In [ ]:
def isPosetMatrix(inputndarray):

    if isBinary(inputndarray) and (np.allclose(inputndarray, np.tril(inputndarray)) or np.allclose(inputndarray, np.triu(inputndarray))):
        mat=np.matmul(inputndarray,inputndarray)
        initial_positions=list(zip(*np.where(inputndarray == 0)))
        updated_positions=list(zip(*np.where( mat == 0)))
        if  updated_positions==initial_positions:
            return True
        else:
            return False
    else:
        return False

**Examples using isPosetMatrix**

In [ ]:
matrix1=np.array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 1, 1]])

In [ ]:
isPosetMatrix(matrix1)

True

In [ ]:
 matrix2= np.array([[1, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],[1, 1, 1, 1]])

In [ ]:
matrix2

array([[1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 0],
       [1, 1, 1, 1]])

In [ ]:
isPosetMatrix(matrix2)

True

In [ ]:
matrix3=np.array([[1,0,0],[1,1,0],[0,1,1]])

In [ ]:
matrix3

array([[1, 0, 0],
       [1, 1, 0],
       [0, 1, 1]])

In [ ]:
isPosetMatrix(matrix3)

False

**findMax** function returns the list of all the maximal elements of a poset matrix.

In [ ]:
def findMax(posetmatrix):

    dimPM=posetmatrix.shape[0]
    maxList=[x for x in range(dimPM) if sum(posetmatrix[:,x])==1]

    return maxList

Example of using **findMax**

In [ ]:
findMax(matrix1)

[1, 2, 4]

**findMin** function returns the list of all the minimal elements of a poset matrix

In [ ]:
def findMin(posetmatrix):

    dimPM=posetmatrix.shape[0]
    minList=[ind for ind in range(dimPM) if sum(posetmatrix[ind])==1 ]


    return minList

Example of using **findMin**

In [ ]:
findMin(matrix1)

[0]

**findMinMax** function returns the list of all the minimal and maximal elements of a poset matrix

In [ ]:
def findMinMax(posetmatrix):

    dimPM=posetmatrix.shape[0]
    minList=[ind for ind in range(dimPM) if sum(posetmatrix[ind])==1 ]
    maxList=[x for x in range(dimPM) if sum(posetmatrix[:,x])==1]

    return (minList,maxList)

In [ ]:
findMinMax(matrix1)

([0], [1, 2, 4])

In [ ]:
findMinMax(matrix2)

([0], [3])

**createU** constructs the submatrix U in a partial composition operation of poset matrices.

In [ ]:
def createU(lposetmatrix,rposetmatrix,i):

    rowAi=lposetmatrix[i-1]
    rowAi=rowAi[0:i-1]
    dimE=rposetmatrix.shape[0]
    U=np.zeros( (dimE,i-1),dtype='int64' )

    for z in range(dimE):
      if z in findMax(rposetmatrix):
        U[z]=rowAi


    return U

**Example of using createU**

In [ ]:
createU(matrix1, matrix2,3)

array([[0, 0],
       [0, 0],
       [0, 0],
       [1, 0]])

**createV** function constructs the submatrix V in a partial composition operation of poset matrices.

In [ ]:
def createV(lposetmatrix,rposetmatrix,i):

   dimF=lposetmatrix.shape[0]
   dimE=rposetmatrix.shape[0]
   colAi=lposetmatrix[i:dimF,i-1]

   v=np.zeros( (dimF-i,dimE),dtype='int64' )
   for k in range(dimE):
      if k in findMin(rposetmatrix):
         v[:,k]=colAi

   return v

Example using **createV**.

In [ ]:
createV(matrix1,matrix2,1)

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [ ]:
createV(matrix2,matrix1,2)

array([[0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

**createUV** function returns the submatrices U and V.

In [ ]:
def createUV(lposetmatrix,rposetmatrix,i):

    dimF=lposetmatrix.shape[0]
    dimE=rposetmatrix.shape[0]
    colAi=lposetmatrix[i:dimF,i-1]
    rowAi=lposetmatrix[i-1]
    rowAi=rowAi[0:i-1]

    V=np.zeros( (dimF-i,dimE),dtype='int64')
    for k in range(dimE):
        if k in findMin(rposetmatrix):
            V[:,k]=colAi

    U=np.zeros( (dimE,i-1),dtype='int64')
    for z in range(dimE):
        if z in findMax(rposetmatrix):
            U[z]=rowAi

    return (V,U)




**Example of using createUV to construct the submatrices U and V.**

In [ ]:
createUV(matrix1,matrix2,2)

(array([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]),
 array([[0],
        [0],
        [0],
        [1]]))

**flattenR(b)** function takes in an ndarray as input and returns a copy of the array collapsed into one dimension

In [ ]:
def flattenR(b):
  return b.flatten()

In [ ]:
def tupletoArray(q):
  size=len(q)
  calc=int(size/(math.sqrt(size)))
  return np.array(q).reshape(calc,calc)

In [ ]:
def totuple(a):
    try:
        return tuple(totuple(i) for i in a)
    except TypeError:
        return a

Set of matrices used for inililiazation of  **counts_list_PM** function.

In [ ]:
rightMat=[np.array([[1, 0],
    [0, 1]]),np.array([[1, 0],
    [1, 1]])]

In [ ]:
leftMat=rightMat

In [ ]:
rightMat

[array([[1, 0],
        [0, 1]]),
 array([[1, 0],
        [1, 1]])]

In [ ]:
leftMat

[array([[1, 0],
        [0, 1]]),
 array([[1, 0],
        [1, 1]])]

In [ ]:
leftMatDisc=[leftMat[0]]

In [ ]:
leftMatDisc

[array([[1, 0],
        [0, 1]])]

In [ ]:
leftMatConn=[leftMat[1]]

In [ ]:
leftMatConn

[array([[1, 0],
        [1, 1]])]

**counts_list_PM** function computes the number of poset matrices or class of poset matrices generated by an operad or partial composition operation.

In [ ]:
def OLDcounts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=str(),second_operation=str(),third_operation=False, n=1):

    counts=[]
    #listPMTypes=list()
    #list_PM=list()
    #num_PMtypes,num_nonPMtypes=list(),list()

    for step in range(n):

        result=list()
        num_PMtypes,num_nonPMtypes=list(),list()
        k=len(listlposetmatrices)

        for item in range(len(listrposetmatrices)):
            rposetmatrix=listrposetmatrices[item]
            computed=list(map(first_operation,listlposetmatrices,[rposetmatrix]*k))
            result.append(computed)

        chainedlist=list(itertools.chain(*result))
        flattened_PMlist=list(itertools.chain(*chainedlist))
        newList=list(map(flattenR,flattened_PMlist))
        tuple_list=totuple(newList)
        uniquetuples=set(tuple_list)
        listlposetmatrices=list(map(tupletoArray, uniquetuples))

        if second_operation!=str():

            tupleTypes=tuple(map(second_operation,listlposetmatrices))
            x=tupleTypes.count(True)
            y=tupleTypes.count(False)
            num_PMtypes.append(x)
            num_nonPMtypes.append(y)

        if third_operation==True:
            #list_PM.append(listlposetmatrices)
            list_PM=list(zip(listlposetmatrices,tupleTypes))
        else:
            list_PM=[]

        counts.append(len(uniquetuples))

    return (counts,list_PM)

In [86]:
def counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=str(),second_operation=str(),third_operation=False, n=1):

    counts=[]
    num_PMtypes,num_nonPMtypes=list(),list()

    for step in range(n):

        result=list()
        k=len(listlposetmatrices)

        for item in range(len(listrposetmatrices)):

            rposetmatrix=listrposetmatrices[item]
            computed=list(map(first_operation,listlposetmatrices,[rposetmatrix]*k))
            result.append(computed)

        chainedlist=list(itertools.chain(*result))
        flattened_PMlist=list(itertools.chain(*chainedlist))
        newList=list(map(flattenR,flattened_PMlist))
        tuple_list=totuple(newList)
        uniquetuples=set(tuple_list)
        listlposetmatrices=list(map(tupletoArray, uniquetuples))

        if second_operation!=str():

            tupleTypes=tuple(map(second_operation,listlposetmatrices))
            x=tupleTypes.count(True)
            y=tupleTypes.count(False)
            num_PMtypes.append(x)
            num_nonPMtypes.append(y)

        if third_operation==True:

            list_PM=list(zip(listlposetmatrices,tupleTypes))

        else:

            list_PM=[]

        counts.append(len(uniquetuples))

    return (counts,list_PM,num_PMtypes, num_nonPMtypes)

**generatePM** function generates poset matrices using the  first operad operation

In [ ]:
def generatePM(lposetmatrix,rposetmatrix):

   dimE=rposetmatrix.shape[0]
   dimF=lposetmatrix.shape[0]
   outputlist=[]

   for i in range(1,(dimF+1)):

      A11=lposetmatrix[0:i-1,0:i-1]
      A21=lposetmatrix[i:dimF,i:dimF]
      A22=lposetmatrix[i:dimF,0:i-1]
      y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
      y2= np.zeros((dimE, dimF-i),dtype='int64' )
      rowAi=lposetmatrix[i-1]
      rowAi=rowAi[0:i-1]
      colAi=lposetmatrix[i:dimF,i-1]
      vecones=np.ones((dimE),dtype='int64' )
      Ui=np.tensordot(vecones, rowAi, 0)
      Vi=np.tensordot(vecones, colAi, 0)
      Vi=np.transpose(Vi)
      block1=np.concatenate((A11, y1), axis=1,dtype='int64')
      block2=np.concatenate((Ui,rposetmatrix,y2), axis=1,dtype='int64')
      block3=np.concatenate((A22,Vi,A21), axis=1,dtype='int64')
      output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
      outputlist.append(output)


   return outputlist

**generatePMMin** function generates the poset matrices using the second operad operation

In [ ]:
def generatePMMin(lposetmatrix,rposetmatrix):

   #rposetmatrix=matrix2
   dimE=rposetmatrix.shape[0]
   dimF=lposetmatrix.shape[0]
   outputlist=[]

   for i in range(1,(dimF+1)):

      A11=lposetmatrix[0:i-1,0:i-1]
      A21=lposetmatrix[i:dimF,i:dimF]
      A22=lposetmatrix[i:dimF,0:i-1]
      y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
      y2= np.zeros((dimE, dimF-i),dtype='int64' )
      rowAi=lposetmatrix[i-1]
      rowAi=rowAi[0:i-1]
      vecones=np.ones((dimE),dtype='int64' )
      Ui=np.tensordot(vecones, rowAi, 0)

      v=createV(lposetmatrix,rposetmatrix,i)
      block1=np.concatenate((A11, y1), axis=1,dtype='int64')
      block2=np.concatenate((Ui,rposetmatrix,y2), axis=1,dtype='int64')
      block3=np.concatenate((A22,v,A21), axis=1,dtype='int64')
      output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
      outputlist.append(output)

   return outputlist

**generatePMMax** function generates the poset matrices using the third operad operation

In [ ]:
def generatePMMax(lposetmatrix,rposetmatrix):

   dimE=rposetmatrix.shape[0]
   dimF=lposetmatrix.shape[0]
   outputlist=[]

   for i in range(1,(dimF+1)):

      A11=lposetmatrix[0:i-1,0:i-1]
      A21=lposetmatrix[i:dimF,i:dimF]
      A22=lposetmatrix[i:dimF,0:i-1]
      y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
      y2= np.zeros((dimE, dimF-i),dtype='int64' )
      U=createU(lposetmatrix,rposetmatrix,i)
      vecones=np.ones((dimE),dtype='int64' )
      colAi=lposetmatrix[i:dimF,i-1]
      Vi=np.tensordot(vecones, colAi, 0)
      Vi=np.transpose(Vi)
      block1=np.concatenate((A11, y1), axis=1,dtype='int64')
      block2=np.concatenate((U,rposetmatrix,y2), axis=1,dtype='int64')
      block3=np.concatenate((A22,Vi,A21), axis=1,dtype='int64')
      output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
      outputlist.append(output)

   return outputlist

**generatePMMinMax** function generates the poset matrices using the partial composition operation defined in terms of the minimal and maximal elements.

In [ ]:
def generatePMMinMax(lposetmatrix,rposetmatrix):

   dimE=rposetmatrix.shape[0]
   dimF=lposetmatrix.shape[0]
   outputlist=[]


   for i in range(1,(dimF+1)):

      A11=lposetmatrix[0:i-1,0:i-1]
      A21=lposetmatrix[i:dimF,i:dimF]
      A22=lposetmatrix[i:dimF,0:i-1]
      y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
      y2= np.zeros((dimE, dimF-i),dtype='int64' )
      U=createU(lposetmatrix,rposetmatrix,i)
      v=createV(lposetmatrix,rposetmatrix,i)
      block1=np.concatenate((A11, y1), axis=1,dtype='int64')
      block2=np.concatenate((U,rposetmatrix,y2), axis=1,dtype='int64')
      block3=np.concatenate((A22,v,A21), axis=1,dtype='int64')
      output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
      outputlist.append(output)

   return outputlist

**Count both Disconnected and Connected Poset Matrices
generated by generatePM**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=str(),third_operation=False,n=8)


([6, 22, 90, 394, 1806, 8558, 41586, 206098], [], [], [])

**Count the number of  Connected Poset Matrices generated using generatePM**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=str(),third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049], [], [], [])

**Count the number of Connected and Disconnected Poset Matrices generated using generatePMMin**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMin,second_operation=str(),third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098], [], [], [])

**Count the number of  Connected Poset Matrices generated using generatePMMin**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMin,second_operation=str(),third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049], [], [], [])

**Count the number of Connected and Disconnected Poset Matrices generated using generatePMMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMax,second_operation=str(),third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098], [], [], [])

**Number of Connected Poset Matrices generated by the operad operation generatePMMax**

In [72]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMax,second_operation=str(),third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049], [], [], [])

**Number of connected and disconnnected poset matrices generated by operad operation generatePMMinMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=str(),third_operation=False,n=8)


([5, 15, 52, 201, 841, 3726, 17213, 82047], [], [], [])

**Number of Connected Poset Matrices generated by the operad operation generatePMMinMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=str(),third_operation=False,n=8)


([2, 6, 22, 90, 394, 1806, 8558, 41586], [], [], [])

**Number of Disconnected Poset Matrices generated by the operad operation generatePMMinMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatDisc,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=str(),third_operation=False,n=8)


([3, 9, 30, 111, 447, 1920, 8655, 40461], [], [], [])

**dualize** function transforms a poset matrix to its dual representation

In [73]:
def dualize(posetmatrix):

  dimPoset=posetmatrix.shape[0]
  dualmatrix = np.zeros((dimPoset,dimPoset),int)

  for s in range(dimPoset):
     for r in range(dimPoset):
        dualmatrix[s,r]=posetmatrix[dimPoset-r-1,dimPoset-s-1]

  return dualmatrix

**typeOfduality** function determines if a poset matrix is self dualizable or non-self dualizable.

In [74]:
def typeOfduality(posetmatrix):

    if np.allclose(dualize(posetmatrix),posetmatrix):
        return False
    else:
        return True

**Examples using dualize and typeOfduality functions**

In [75]:
matrix1

array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 1, 1]])

In [76]:
dualize(matrix1)

array([[1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 1, 1, 1, 1]])

In [79]:
matrix2

array([[1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 0],
       [1, 1, 1, 1]])

In [80]:
dualize(matrix2)

array([[1, 0, 0, 0],
       [1, 1, 0, 0],
       [1, 0, 1, 0],
       [1, 1, 1, 1]])

In [81]:
matrix4=np.array([[1,0],[1,1]])

In [82]:
 typeOfduality(matrix4)

False

In [83]:
 typeOfduality(matrix1)

True

**Enumeration of Connected and Disconnected poset matrices generated using generatePM in terms of self dual and non-self dual types.**

In [84]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=typeOfduality,third_operation=True,n=3)

([6, 22, 90],
 [(array([[1, 0, 0, 0, 0],
          [0, 1, 0, 0, 0],
          [0, 0, 1, 0, 0],
          [0, 0, 1, 1, 0],
          [0, 0, 0, 0, 1]]),
   True),
  (array([[1, 0, 0, 0, 0],
          [1, 1, 0, 0, 0],
          [1, 1, 1, 0, 0],
          [0, 0, 0, 1, 0],
          [1, 1, 1, 1, 1]]),
   True),
  (array([[1, 0, 0, 0, 0],
          [1, 1, 0, 0, 0],
          [1, 1, 1, 0, 0],
          [1, 1, 0, 1, 0],
          [1, 1, 0, 1, 1]]),
   True),
  (array([[1, 0, 0, 0, 0],
          [0, 1, 0, 0, 0],
          [1, 1, 1, 0, 0],
          [0, 0, 0, 1, 0],
          [1, 1, 1, 1, 1]]),
   True),
  (array([[1, 0, 0, 0, 0],
          [1, 1, 0, 0, 0],
          [1, 1, 1, 0, 0],
          [1, 1, 1, 1, 0],
          [0, 0, 0, 0, 1]]),
   True),
  (array([[1, 0, 0, 0, 0],
          [0, 1, 0, 0, 0],
          [0, 1, 1, 0, 0],
          [0, 1, 1, 1, 0],
          [1, 1, 1, 1, 1]]),
   True),
  (array([[1, 0, 0, 0, 0],
          [1, 1, 0, 0, 0],
          [1, 1, 1, 0, 0],
          [1, 0, 0, 1, 

In [88]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=typeOfduality,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 [4, 16, 84, 372, 1784, 8468, 41496, 205704],
 [2, 6, 6, 22, 22, 90, 90, 394])

**Enumeration of Connected poset matrices generated using generatePM in terms of self dual and non-self dual types.**

In [89]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=typeOfduality,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 [2, 8, 42, 186, 892, 4234, 20748, 102852],
 [1, 3, 3, 11, 11, 45, 45, 197])

**Enumeration of Connected and Disconnected poset matrices generated using generatePMMin in terms of self dual and non-self dual types.**

In [90]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMin,second_operation=typeOfduality,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 [4, 15, 84, 372, 1788, 8493, 41533, 205909],
 [2, 7, 6, 22, 18, 65, 53, 189])

**Enumeration of Connected poset matrices generated using generatePMMin in terms of self dual and non-self dual types.**

In [91]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMin,second_operation=typeOfduality,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 [2, 7, 42, 187, 896, 4257, 20778, 103003],
 [1, 4, 3, 10, 7, 22, 15, 46])

**Enumeration of Connected and Disconnected poset matrices generated using generatePMMax in terms of self dual and non-self dual types.**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMax,second_operation=typeOfduality,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 ([4, 15, 84, 372, 1788, 8493, 41533, 205909], [2, 7, 6, 22, 18, 65, 53, 189]))

**Enumeration of Connected poset matrices generated using generatePMMax in terms of self dual and non-self dual types.**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMax,second_operation=typeOfduality,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 ([2, 7, 42, 187, 896, 4257, 20778, 103003], [1, 4, 3, 10, 7, 22, 15, 46]))

**Enumeration of Connected and Disconnected poset matrices generated using generatePMMinMax in terms of self dual and non-self dual types.**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=typeOfduality,third_operation=False,n=8)

([5, 15, 52, 201, 841, 3726, 17213, 82047],
 [],
 ([4, 10, 50, 186, 836, 3674, 17198, 81846], [1, 5, 2, 15, 5, 52, 15, 201]))

**Enumeration of Connected poset matrices generated using generatePMMinMax in terms of self dual and non-self dual types.**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=typeOfduality,third_operation=False,n=8)

([2, 6, 22, 90, 394, 1806, 8558, 41586],
 [],
 ([2, 4, 22, 84, 394, 1784, 8558, 41496], [0, 2, 0, 6, 0, 22, 0, 90]))

**Enumeration of Disonnected poset matrices generated using generatePMMinMax in terms of self dual and non-self dual types.**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatDisc,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=typeOfduality,third_operation=False,n=8)

([3, 9, 30, 111, 447, 1920, 8655, 40461],
 [],
 ([2, 6, 28, 102, 442, 1890, 8640, 40350], [1, 3, 2, 9, 5, 30, 15, 111]))

**Enumeration of poset matrices here is based on the partial composition operations defined in Theorem 2.1 of the paper "Operad Structure of Poset Matrices."**

In [92]:
def partialComp1(lposetmatrix,rposetmatrix):

   dimE=rposetmatrix.shape[0]
   dimF=lposetmatrix.shape[0]
   outputlist=[]

   for i in range(1,(dimF+1)):

      A11=lposetmatrix[0:i-1,0:i-1]
      A21=lposetmatrix[i:dimF,i:dimF]
      A22=lposetmatrix[i:dimF,0:i-1]
      y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
      y2= np.zeros((dimE, dimF-i),dtype='int64' )
      Ui=np.ones((dimE,i-1),dtype='int64' )
      Vi=np.ones((dimF-i,  dimE),dtype='int64' )
      block1=np.concatenate((A11, y1), axis=1,dtype='int64')
      block2=np.concatenate((Ui,rposetmatrix,y2), axis=1,dtype='int64')
      block3=np.concatenate((A22,Vi,A21), axis=1,dtype='int64')
      output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
      outputlist.append(output)


   return outputlist

In [93]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=partialComp1,second_operation=str(),third_operation=False,n=15)

([3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584],
 [],
 [],
 [])

In [ ]:
def partialComp2(lposetmatrix,rposetmatrix):

    dimE=rposetmatrix.shape[0]
    dimF=lposetmatrix.shape[0]
    outputlist=[]

    for i in range(1,(dimF+1)):
        A11=lposetmatrix[0:i-1,0:i-1]
        A21=lposetmatrix[i:dimF,i:dimF]
        A22=np.ones((dimF-i,i-1),dtype='int64' )
        y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
        y2= np.zeros((dimE, dimF-i),dtype='int64' )
        Ui=np.zeros((dimE,i-1),dtype='int64' )
        Vi=np.zeros((dimF-i,  dimE),dtype='int64' )
        block1=np.concatenate((A11, y1), axis=1,dtype='int64')
        block2=np.concatenate((Ui,rposetmatrix,y2), axis=1,dtype='int64')
        block3=np.concatenate((A22,Vi,A21), axis=1,dtype='int64')
        output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
        outputlist.append(output)

    return outputlist

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=partialComp2,second_operation=str(),third_operation=False,n=8)

([3, 6, 16, 40, 112, 314, 900, 2634], [], ([], []))

In [ ]:
def partialComp3(lposetmatrix,rposetmatrix):

    dimE=rposetmatrix.shape[0]
    dimF=lposetmatrix.shape[0]
    outputlist=[]

    for i in range(1,(dimF+1)):

        A11=lposetmatrix[0:i-1,0:i-1]
        A21=lposetmatrix[i:dimF,i:dimF]
        A22=np.ones((dimF-i,i-1),dtype='int64' )
        y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
        y2= np.zeros((dimE, dimF-i),dtype='int64' )
        Ui=np.ones((dimE,i-1),dtype='int64' )
        Vi=np.zeros((dimF-i,  dimE),dtype='int64' )
        block1=np.concatenate((A11, y1), axis=1,dtype='int64')
        block2=np.concatenate((Ui,rposetmatrix,y2), axis=1,dtype='int64')
        block3=np.concatenate((A22,Vi,A21), axis=1,dtype='int64')
        output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
        outputlist.append(output)

    return outputlist

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=partialComp3,second_operation=str(),third_operation=False,n=8)

([4, 10, 28, 80, 244, 774, 2524, 8414], [], ([], []))

In [ ]:
def partialComp4(lposetmatrix,rposetmatrix):

    dimE=rposetmatrix.shape[0]
    dimF=lposetmatrix.shape[0]
    outputlist=[]


    for i in range(1,(dimF+1)):

        A11=lposetmatrix[0:i-1,0:i-1]
        A21=lposetmatrix[i:dimF,i:dimF]
        A22=np.ones((dimF-i,i-1),dtype='int64' )
        y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
        y2= np.zeros((dimE, dimF-i),dtype='int64' )
        Ui=np.zeros((dimE,i-1),dtype='int64' )
        Vi=np.ones((dimF-i,  dimE),dtype='int64' )
        block1=np.concatenate((A11, y1), axis=1,dtype='int64')
        block2=np.concatenate((Ui,rposetmatrix,y2), axis=1,dtype='int64')
        block3=np.concatenate((A22,Vi,A21), axis=1,dtype='int64')
        output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
        outputlist.append(output)

    return outputlist

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=partialComp4,second_operation=str(),third_operation=False,n=8)

([4, 10, 28, 80, 244, 774, 2524, 8414], [], ([], []))

In [ ]:
def partialComp5(lposetmatrix,rposetmatrix):

    dimE=rposetmatrix.shape[0]
    dimF=lposetmatrix.shape[0]
    outputlist=[]

    for i in range(1,(dimF+1)):

        A11=lposetmatrix[0:i-1,0:i-1]
        A21=lposetmatrix[i:dimF,i:dimF]
        A22=np.zeros((dimF-i,i-1),dtype='int64' )
        y1=np.zeros( (i-1, dimF+dimE-i),dtype='int64' )
        y2= np.zeros((dimE, dimF-i),dtype='int64' )
        Ui=np.zeros((dimE,i-1),dtype='int64' )
        Vi=np.zeros((dimF-i,  dimE),dtype='int64' )

        block1=np.concatenate((A11, y1), axis=1,dtype='int64')
        block2=np.concatenate((Ui,rposetmatrix,y2), axis=1,dtype='int64')
        block3=np.concatenate((A22,Vi,A21), axis=1,dtype='int64')
        output=np.concatenate((block1,block2,block3), axis=0,dtype='int64')
        outputlist.append(output)

    return outputlist

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=partialComp5,second_operation=str(),third_operation=False,n=14)

([3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597], [], ([], []))

**Test for Connectivity of Poset Matrices**

In [94]:
def getEltPair(pair,flattenedlistOfPairs):

    setp=set()
    element1=pair[0]
    element2=pair[1]

    indiceslist=[i for i, x in enumerate(flattenedlistOfPairs) if x == element1]+[i for i, x in enumerate(flattenedlistOfPairs) if x == element2]

    for ind in indiceslist:

        if ind%2==0:
            setp.add(flattenedlistOfPairs[ind+1])
        else:
            setp.add(flattenedlistOfPairs[ind-1])

    return setp

In [95]:
def isConnected(posetmatrix):

    dimposetmatrix=posetmatrix.shape[0]
    positions_ones=list(zip(*np.where(posetmatrix == 1)))
    new_positions_ones=[io for io in positions_ones if io[0]-io[1]!=0]

    if len( new_positions_ones)==0:
        return False

    list1=[new_positions_ones[0][0],new_positions_ones[0][1]]
    set1=set(list1)
    list2=new_positions_ones[1:]

    for t in range(len(list2)):

        commonSet=set1.intersection(set(list2[t]))
        flattenedlist1=list(itertools.chain(*list2[t+1:]))
        setpts=getEltPair(list2[t],flattenedlist1)
        commonSet2=set1.intersection(setpts)

        if len(commonSet)>0 or len(commonSet2)>0:
            set1=set1.union(set(list2[t]))
        else:
            return False

    if len(set1)==dimposetmatrix:
        return True
    else:
        return False

**isSemiRight** function determines if a poset matrix is semi-right dualizable.

In [96]:
def isSemiRight(posetmatrix):

    dimposetmatrix=posetmatrix.shape[0]
    i,depth1=0,0
    while i<dimposetmatrix:
        if  all(posetmatrix[i:,i]):
            depth1=depth1+1
            i=i+1
        else:
            break
    if (depth1>0 and not isConnected(posetmatrix[depth1:,depth1:]) and not np.allclose(dualize(posetmatrix[depth1:,depth1:]),posetmatrix[depth1:,depth1:])):
        return True
    else:
        return False

**isSemiLeft** function determines if a poset matrix is semi-left dualizable.

In [97]:
def isSemiLeft(posetmatrix):

    dimposetmatrix=posetmatrix.shape[0]
    i,depth2=0,0

    while i<dimposetmatrix:
        if  all(posetmatrix[dimposetmatrix-i-1,:dimposetmatrix-i]):
            depth2=depth2+1
            i=i+1
        else:
            break
    else:
        depth2
    if (depth2>0 and not isConnected(posetmatrix[:-depth2,:-depth2]) and not np.allclose(dualize(posetmatrix[:-depth2,:-depth2]),posetmatrix[:-depth2,:-depth2])):
        return  True
    else:
        return False

**isSemiEquidual** function determines if a poset matrix is semi-equidual

In [98]:
def isSemiEquidual(posetmatrix):
    if isSemiLeft(posetmatrix) or isSemiRight(posetmatrix):
        return True
    else:
        return False

**Enumeration of poset matrices for the operations isSemiRight and generatePM**

In [99]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=isSemiRight,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 [0, 2, 10, 53, 250, 1224, 5982, 29853],
 [6, 20, 80, 341, 1556, 7334, 35604, 176245])

**Enumeration of poset matrices corresponding to the operations isSemiRight and generatePMMin**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMin,second_operation=isSemiRight,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 ([0, 2, 10, 53, 249, 1223, 5984, 29863],
  [6, 20, 80, 341, 1557, 7335, 35602, 176235]))

**Enumeration of poset matrices corresponding to operations isSemiRight and generatePMMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMax,second_operation=isSemiRight,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 ([0, 0, 0, 0, 0, 0, 0, 0], [6, 22, 90, 394, 1806, 8558, 41586, 206098]))

**Enumeration of poset matrices corresponding to operations isSemiRight and generatePMMinMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=isSemiRight,third_operation=False,n=8)

([5, 15, 52, 201, 841, 3726, 17213, 82047],
 [],
 ([0, 0, 0, 0, 0, 0, 0, 0], [5, 15, 52, 201, 841, 3726, 17213, 82047]))

**Enumeration of poset matrices for the operations isSemiLeft and generatePM**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=isSemiLeft,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 ([0, 2, 10, 52, 239, 1144, 5487, 27011],
  [6, 20, 80, 342, 1567, 7414, 36099, 179087]))

**Enumeration of poset matrices for the operations isSemiLeft and generatePMMin**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMin,second_operation=isSemiLeft,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 ([0, 0, 0, 0, 0, 0, 0, 0], [6, 22, 90, 394, 1806, 8558, 41586, 206098]))

**Enumeration of poset matrices for the operations isSemiLeft and generatePMMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMax,second_operation=isSemiLeft,third_operation=False,n=8)

([6, 22, 90, 394, 1806, 8558, 41586, 206098],
 [],
 ([0, 2, 10, 53, 251, 1255, 6295, 32277],
  [6, 20, 80, 341, 1555, 7303, 35291, 173821]))

**Enumeration of poset matrices for the operations isSemiLeft and generatePMMinMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMat,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=isSemiLeft,third_operation=False,n=8)

([5, 15, 52, 201, 841, 3726, 17213, 82047],
 [],
 ([0, 0, 0, 0, 0, 0, 0, 0], [5, 15, 52, 201, 841, 3726, 17213, 82047]))

**Enumeration of poset matrices for the operations isSemiEquiDual and generatePM**

In [ ]:
a1=np.array([0, 2, 10, 53, 250, 1224, 5982, 29853])

In [ ]:
a2=np.array([0, 2, 10, 52, 239, 1144, 5487, 27011])

In [ ]:
np.add(a1,a2)

array([    0,     4,    20,   105,   489,  2368, 11469, 56864])

**Enumeration of poset matrices for the operations isSemiEquiDual and generatePMMin**

In [ ]:
b1=np.array([0, 2, 10, 53, 249, 1223, 5984, 29863])

In [ ]:
b2=np.array([0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
np.add(b1,b2)

array([    0,     2,    10,    53,   249,  1223,  5984, 29863])

**Enumeration of poset matrices for the operations isSemiEquiDual and generatePMMax**

In [ ]:
c1=np.array([0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
c2=np.array([0, 2, 10, 53, 251, 1255, 6295, 32277])

In [ ]:
np.add(c1,c2)

array([    0,     2,    10,    53,   251,  1255,  6295, 32277])

**Enumeration of poset matrices for the operations isSemiEquiDual and generatePMMinMax**

In [ ]:
d1=np.array([0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
d2=np.array([0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
np.add(d1,d2)

array([0, 0, 0, 0, 0, 0, 0, 0])

In [100]:
def isOrdinalSum(posetmatrix):

    level,column=0,0
    dimPosetMatrix=posetmatrix.shape[0]

    while posetmatrix[-1,column]==1 and posetmatrix[-2,column]==1 and dimPosetMatrix>2:
        column=column+1
        if column>0:
            needarray=np.ones(( column,), dtype=int)
        else:
            column
            break

    while  posetmatrix[-1,column]==1 and (posetmatrix[-2,0]!=1 or dimPosetMatrix==2):

        column=column+1
        if column==dimPosetMatrix:
            break

    if 'needarray' in vars():

        if len(needarray)!=0:
            level=level+2
        else:
            level

        for s in range(dimPosetMatrix-3,-1,-1):
            if len(needarray)>0 and np.array_equal( needarray,posetmatrix[s,0:column]):
                level=level+1
            else:
                level
                break
        if level==dimPosetMatrix:
            level=level-1


    if column>1 and level>1:
            column=column-1


    return column+level==dimPosetMatrix

In [101]:
def isDirectSum(posetmatrix):

    level,column=0,0
    dimPosetMatrix=posetmatrix.shape[0]

    while posetmatrix[-1,column]==0 and posetmatrix[-2,column]==0 and dimPosetMatrix>2:
        column=column+1
        if column>0:
            needarray=np.zeros(( column,), dtype=int)
        else:
            column
            break

    while  posetmatrix[-1,column]==0 and (posetmatrix[-2,0]!=0 or dimPosetMatrix==2):


        if dimPosetMatrix>2:
            column=column+1
            level=1
        else:
            column=2

        if column==dimPosetMatrix:
            break

    if 'needarray' in vars():

        if len(needarray)!=0:
            level=level+2
        else:
            level

        for s in range(dimPosetMatrix-3,-1,-1):
            if len(needarray)>0 and np.array_equal( needarray,posetmatrix[s,0:column]):
                level=level+1
            else:
                level
                break
        if level==dimPosetMatrix:
            level=level-1

    return column+level==dimPosetMatrix

**Enumeration of poset matrices for the operations isOrdinalSum and generatePM**

In [102]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=isOrdinalSum,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 [3, 10, 37, 151, 659, 3013, 14249, 69131],
 [0, 1, 8, 46, 244, 1266, 6544, 33918])

**Enumeration of poset matrices for the operations isOrdinalSum and generatePMMin**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMin,second_operation=isOrdinalSum,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 ([3, 8, 25, 91, 367, 1591, 7263, 34415],
  [0, 3, 20, 106, 536, 2688, 13530, 68634]))

**Enumeration of poset matrices for the operations isOrdinalSum and generatePMMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMax,second_operation=isOrdinalSum,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 ([3, 8, 23, 81, 329, 1451, 6723, 32189],
  [0, 3, 22, 116, 574, 2828, 14070, 70860]))

**Enumeration of poset matrices for the operations isOrdinalSum and generatePMMinMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=isOrdinalSum,third_operation=False,n=8)

([2, 6, 22, 90, 394, 1806, 8558, 41586],
 [],
 ([2, 2, 2, 2, 2, 2, 2, 2], [0, 4, 20, 88, 392, 1804, 8556, 41584]))

**reverseArray** reverses an array

In [ ]:
def reverseArray(array):
    dimArray=len(array)
    new_array=np.array([])
    for item in range(dimArray):
         new_array=np.append(new_array,array[dimArray-1-item])

    return new_array.astype(int)

**isDoubleDual** function determines if a poset matrix is double dualizable.

In [ ]:
def isDoubleDual(posetmatrix):

    last_row=posetmatrix[-1]
    first_col=posetmatrix[:,0]

    if np.allclose(last_row,reverseArray(first_col)) and len(findMax(posetmatrix))==len(findMin(posetmatrix)):
        x=posetmatrix[:-1,:-1]
        y=posetmatrix[1:,1:]

        if isSemiRight(x) and isSemiLeft(y) and np.allclose(x[1:,1:],y[:-1,:-1]):
            return True
        else:
            return False
    else:
        return False

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePM,second_operation=isDoubleDual,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 ([0, 0, 2, 8, 42, 195, 966, 4744], [3, 11, 43, 189, 861, 4084, 19827, 98305]))

**Enumeration of poset matrices for the operations isDoubleDual and generatePMMin**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMin,second_operation=isDoubleDual,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 ([0, 0, 0, 0, 0, 0, 0, 0], [3, 11, 45, 197, 903, 4279, 20793, 103049]))

**Enumeration of poset matrices for the operations isDoubleDual and generatePMMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMax,second_operation=isDoubleDual,third_operation=False,n=8)

([3, 11, 45, 197, 903, 4279, 20793, 103049],
 [],
 ([0, 0, 0, 0, 0, 0, 0, 0], [3, 11, 45, 197, 903, 4279, 20793, 103049]))

**Enumeration of poset matrices for the operations isDoubleDual and generatePMMinMax**

In [ ]:
counts_list_PM(listlposetmatrices=leftMatConn,listrposetmatrices=rightMat,first_operation=generatePMMinMax,second_operation=isDoubleDual,third_operation=False,n=8)

([2, 6, 22, 90, 394, 1806, 8558, 41586],
 [],
 ([0, 0, 0, 0, 0, 0, 0, 0], [2, 6, 22, 90, 394, 1806, 8558, 41586]))